In [12]:
from docx import Document
from docx2pdf import convert
from transformers import pipeline
import sys

# Hugging Face pipeline'ını başlat (örneğin, T5-small modeli kullanıyoruz)
def initialize_text_generator():
    try:
        text_generator = pipeline("text-generation", model="facebook/bart-large")
        return text_generator
    except Exception as e:
        print(f"Model yüklenirken hata oluştu: {e}")
        sys.exit(1)

# Metin üretme fonksiyonu
def generate_content_with_huggingface(text_generator, prompt):
    try:
        generated_texts = text_generator(prompt, max_length=500)
        return generated_texts[0]['generated_text']
    except Exception as e:
        print(f"Metin üretilirken hata oluştu: {e}")
        return None

# Kullanıcı onayı alma fonksiyonu
def ask_user_for_confirmation(content, item_type="içerik"):
    print(f"\nLLM tarafından üretilen {item_type}:\n", content)
    user_input = input(f"Bu {item_type} belgeye eklemek istiyor musunuz? (okay/hayır): ").strip().lower()
    return user_input == "okay"

# Tablo ekleme fonksiyonu
def add_table_to_document(document, table_data):
    try:
        table = document.add_table(rows=1, cols=len(table_data[0]))
        hdr_cells = table.rows[0].cells
        for i, header in enumerate(table_data[0]):
            hdr_cells[i].text = header.strip()
        for row in table_data[1:]:
            row_cells = table.add_row().cells
            for i, cell in enumerate(row):
                row_cells[i].text = cell.strip()
        print("Tablo belgeye eklendi.")
    except Exception as e:
        print(f"Tablo eklenirken hata oluştu: {e}")

# İçeriği analiz edip belgeye uygun şekilde ekleme fonksiyonu
def add_content_to_document(document, content):
    if not content:
        print("İçerik bulunamadı, belgeye ekleme yapılamadı.")
        return

    lines = content.split('\n')
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        if line.lower().startswith("başlık:"):
            title = line.replace("başlık:", "").strip()
            if ask_user_for_confirmation(title, "başlık"):
                document.add_heading(title, level=1)
                print(f"Başlık eklendi: {title}")
        elif line.lower().startswith("tablo:"):
            table_data = []
            i += 1
            while i < len(lines) and lines[i].strip() and not lines[i].strip().lower().startswith(("başlık:", "tablo:")):
                table_data.append(lines[i].strip().split(';'))
                i += 1
            if ask_user_for_confirmation(table_data, "tablo"):
                add_table_to_document(document, table_data)
            continue  # while döngüsünde i zaten artırıldı
        else:
            if line and ask_user_for_confirmation(line, "paragraf"):
                document.add_paragraph(line)
                print(f"Paragraf eklendi: {line}")
        i += 1

# Agent: Word belgesi oluşturma ve PDF'ye dönüştürme fonksiyonu
def create_document_and_convert_to_pdf(content, output_word_file, output_pdf_file):
    try:
        # Word belgesi oluştur
        document = Document()
        document.add_heading('Generated Document', 0)

        # İçeriği belgeye ekle
        add_content_to_document(document, content)

        # Word belgesini kaydet
        document.save(output_word_file)
        print(f"Word belgesi '{output_word_file}' oluşturuldu.")

        # Word belgesini PDF'ye dönüştür
        convert(output_word_file, output_pdf_file)
        print(f"PDF dosyası '{output_pdf_file}' oluşturuldu.")
    except Exception as e:
        print(f"Belge oluşturulurken veya PDF'ye dönüştürülürken hata oluştu: {e}")

# Ana işlem
if __name__ == "__main__":
    # Modeli başlat
    text_generator = initialize_text_generator()

    # Hugging Face ile içerik üret
    prompt = "Explain the impact of blockchain technology on the financial industry."
    generated_content = generate_content_with_huggingface(text_generator, prompt)

    if generated_content:
        # Agent: Belgeyi oluştur ve PDF'ye dönüştür
        output_word_file = "generated_document_hf.docx"
        output_pdf_file = "generated_document_hf.pdf"
        create_document_and_convert_to_pdf(generated_content, output_word_file, output_pdf_file)
    else:
        print("İçerik üretilemedi, işlem sonlandırıldı.")

config.json:   0%|          | 0.00/1.63k [00:00<?, ?B/s]

c:\Users\emreq\Desktop\Genel\.venv\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\emreq\.cache\huggingface\hub\models--facebook--bart-large. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


pytorch_model.bin:   0%|          | 0.00/1.02G [00:00<?, ?B/s]

KeyboardInterrupt: 

In [17]:
from docx import Document
from docx2pdf import convert
from transformers import pipeline
import sys

# Hugging Face pipeline'ını başlat
def initialize_text_generator():
    try:
        text_generator = pipeline("text-generation", model="gpt2")
        return text_generator
    except Exception as e:
        print(f"Model yüklenirken hata oluştu: {e}")
        sys.exit(1)

# Metin üretme fonksiyonu
def generate_content_with_huggingface(text_generator, prompt):
    try:
        generated_texts = text_generator(prompt, max_length=200, num_return_sequences=1)
        return generated_texts[0]['generated_text']
    except Exception as e:
        print(f"Metin üretilirken hata oluştu: {e}")
        return None

# Tablo ekleme fonksiyonu
def add_table_to_document(document, table_data):
    try:
        table = document.add_table(rows=1, cols=len(table_data[0]))
        hdr_cells = table.rows[0].cells
        for i, header in enumerate(table_data[0]):
            hdr_cells[i].text = header.strip()
        for row in table_data[1:]:
            row_cells = table.add_row().cells
            for i, cell in enumerate(row):
                row_cells[i].text = cell.strip()
        print("Tablo belgeye eklendi.")
    except Exception as e:
        print(f"Tablo eklenirken hata oluştu: {e}")

# İçeriği analiz edip belgeye uygun şekilde ekleme fonksiyonu
def add_content_to_document(document, content):
    if not content:
        print("İçerik bulunamadı, belgeye ekleme yapılamadı.")
        return

    lines = content.split('\n')
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        if line.lower().startswith("başlık:"):
            # Başlık ekle
            title = line.replace("başlık:", "").strip()
            document.add_heading(title, level=1)
            print(f"Başlık eklendi: {title}")
        elif line.lower().startswith("tablo:"):
            # Tablo ekle
            table_data = []
            i += 1
            while i < len(lines) and lines[i].strip() and not lines[i].strip().lower().startswith(("başlık:", "tablo:")):
                table_data.append(lines[i].strip().split(';'))
                i += 1
            if table_data:
                add_table_to_document(document, table_data)
            continue  # while döngüsünde i zaten artırıldı
        else:
            # Paragraf ekle
            if line:
                document.add_paragraph(line)
                print(f"Paragraf eklendi: {line}")
        i += 1

# Belge oluşturma ve PDF'ye dönüştürme fonksiyonu
def create_document_and_convert_to_pdf(content, output_word_file, output_pdf_file):
    try:
        # Word belgesi oluştur
        document = Document()
        document.add_heading('Generated Document', 0)

        # İçeriği belgeye ekle
        add_content_to_document(document, content)

        # Word belgesini kaydet
        document.save(output_word_file)
        print(f"Word belgesi '{output_word_file}' oluşturuldu.")

        # Word belgesini PDF'ye dönüştür
        try:
            convert(output_word_file, output_pdf_file)
            print(f"PDF dosyası '{output_pdf_file}' oluşturuldu.")
        except Exception as e:
            print(f"PDF dönüştürme sırasında hata oluştu: {e}")
    except Exception as e:
        print(f"Belge oluşturulurken hata oluştu: {e}")

# Ana işlem
if __name__ == "__main__":
    # Modeli başlat
    text_generator = initialize_text_generator()

    # Hugging Face ile içerik üret
    prompt =""" Başlık: Impact of Blockchain on Financial Industry
Tablo:
Advantage; Challenge
Transparency; Scalability
Security; Regulatory issues
Decentralization; High energy consumption"""

    generated_content = generate_content_with_huggingface(text_generator, prompt)

    if generated_content:
        # Belgeyi oluştur ve PDF'ye dönüştür
        output_word_file = "generated_document_hf.docx"
        output_pdf_file = "generated_document_hf.pdf"
        create_document_and_convert_to_pdf(generated_content, output_word_file, output_pdf_file)
    else:
        print("İçerik üretilemedi, işlem sonlandırıldı.")


Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Başlık eklendi: Başlık: Impact of Blockchain on Financial Industry
Tablo belgeye eklendi.
Word belgesi 'generated_document_hf.docx' oluşturuldu.


  0%|          | 0/1 [00:00<?, ?it/s]

PDF dosyası 'generated_document_hf.pdf' oluşturuldu.


In [ ]:
from docx import Document
from docx.shared import Pt
from docx.enum.style import WD_STYLE_TYPE
from docx2pdf import convert
from transformers import pipeline
import sys
import os

def initialize_text_generator():
    try:
        text_generator = pipeline("text-generation", model="EleutherAI/gpt-neo-1.3B")
        print("Metin üretme modeli başarıyla yüklendi.")
        return text_generator
    except Exception as e:
        print(f"Model yüklenirken hata oluştu: {e}")
        sys.exit(1)

def generate_content_with_huggingface(text_generator, prompt):
    try:
        generated_texts = text_generator(prompt, max_length=1000, num_return_sequences=1)
        return generated_texts[0]['generated_text']
    except Exception as e:
        print(f"Metin üretilirken hata oluştu: {e}")
        return None

def add_styled_table(document, table_data, style="Table Grid"):
    try:
        if not table_data or not table_data[0]:
            print("Tablo verisi eksik, tablo eklenemedi.")
            return

        table = document.add_table(rows=1, cols=len(table_data[0]))
        table.style = style

        hdr_cells = table.rows[0].cells
        for i, header in enumerate(table_data[0]):
            hdr_cells[i].text = header.strip()

        for row in table_data[1:]:
            row_cells = table.add_row().cells
            for i, cell in enumerate(row):
                row_cells[i].text = cell.strip()

        print("Stil uygulanmış tablo belgeye eklendi.")
    except Exception as e:
        print(f"Tablo eklenirken hata oluştu: {e}")

def parse_table_data(table_string):
    try:
        if not table_string:
            print("Tablo verisi boş.")
            return []

        table_lines = table_string.split(';')
        table_data = [line.split(',') for line in table_lines if line]
        if not all(len(row) == len(table_data[0]) for row in table_data):
            print("Tablo verisi sütun sayısı açısından tutarsız.")
            return []

        return table_data
    except Exception as e:
        print(f"Tablo ayrıştırılırken hata oluştu: {e}")
        return []

def add_content_to_document(document, content):
    if not content:
        print("İçerik bulunamadı, belgeye ekleme yapılamadı.")
        return

    # Define custom styles
    styles = document.styles
    title_style = styles.add_style('CustomTitle', WD_STYLE_TYPE.PARAGRAPH)
    title_style.font.size = Pt(24)
    title_style.font.bold = True

    heading_style = styles.add_style('CustomHeading', WD_STYLE_TYPE.PARAGRAPH)
    heading_style.font.size = Pt(18)
    heading_style.font.bold = True

    body_style = styles.add_style('CustomBody', WD_STYLE_TYPE.PARAGRAPH)
    body_style.font.size = Pt(11)

    lines = content.split('\n')
    for line in lines:
        line = line.strip()
        if line.lower().startswith("başlık:"):
            title = line.replace("başlık:", "").strip()
            document.add_paragraph(title, style='CustomTitle')
            print(f"Başlık eklendi: {title}")
        elif line.lower().startswith("altbaşlık:"):
            subtitle = line.replace("altbaşlık:", "").strip()
            document.add_paragraph(subtitle, style='CustomHeading')
            print(f"Alt başlık eklendi: {subtitle}")
        elif line.lower().startswith("tablo:"):
            table_string = line.replace("tablo:", "").strip()
            table_data = parse_table_data(table_string)
            if table_data:
                add_styled_table(document, table_data, style="Table Grid")
        elif line:
            paragraph = document.add_paragraph(line, style='CustomBody')
            print(f"Paragraf eklendi: {line[:50]}...")

def create_document_and_convert_to_pdf(content, output_word_file, output_pdf_file):
    try:
        document = Document()
        document.add_heading('Generated Document', 0)

        add_content_to_document(document, content)

        document.save(output_word_file)
        print(f"Word belgesi '{output_word_file}' oluşturuldu.")

        if os.name == 'nt':  # Windows
            try:
                convert(output_word_file, output_pdf_file)
                print(f"PDF dosyası '{output_pdf_file}' oluşturuldu.")
            except Exception as e:
                print(f"PDF dönüştürme sırasında hata oluştu: {e}")
        else:
            try:
                import pypandoc
                pypandoc.convert_file(output_word_file, 'pdf', outputfile=output_pdf_file)
                print(f"PDF dosyası '{output_pdf_file}' oluşturuldu (pypandoc ile).")
            except Exception as e:
                print(f"PDF dönüştürme sırasında hata oluştu (alternatif): {e}")
    except Exception as e:
        print(f"Belge oluşturulurken hata oluştu: {e}")

if __name__ == "__main__":
    text_generator = initialize_text_generator()

    prompt = (
        "Başlık: Blockchain'in Finans Sektörüne Etkisi\n"
        "Altbaşlık: Avantajlar ve Zorluklar\n"
        "Explain the impact of blockchain technology on the financial industry. "
        "Discuss the advantages and challenges.\n"
        "Tablo: Avantaj,Zorluk;Şeffaflık,Ölçeklenebilirlik;Güvenlik,Düzenleyici sorunlar;Merkeziyetsizlik,Yüksek enerji tüketimi"
    )
    generated_content = generate_content_with_huggingface(text_generator, prompt)

    if generated_content:
        output_word_file = "generated_document_hf.docx"
        output_pdf_file = "generated_document_hf.pdf"
        create_document_and_convert_to_pdf(generated_content, output_word_file, output_pdf_file)
    else:
        print("İçerik üretilemedi, işlem sonlandırıldı.")

Device set to use cpu


Metin üretme modeli başarıyla yüklendi.


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [22]:
from docx import Document
from docx2pdf import convert
from transformers import pipeline
import sys

def initialize_text_generator():
    try:
        text_generator = pipeline("text-generation", model="gpt2")
        print("Metin üretme modeli başarıyla yüklendi.")
        return text_generator
    except Exception as e:
        print(f"Model yüklenirken hata oluştu: {e}")
        sys.exit(1)

def generate_content_with_huggingface(text_generator, prompt, max_length=200):
    try:
        generated_texts = text_generator(prompt, max_length=max_length, num_return_sequences=1)
        generated_content = generated_texts[0]['generated_text']
        print("İçerik başarıyla oluşturuldu.")
        print("Oluşturulan içerik:")
        print(generated_content)
        return generated_content
    except Exception as e:
        print(f"Metin üretilirken hata oluştu: {e}")
        return None

def add_table_to_document(document, table_data):
    if not table_data:
        print("Tablo verisi sağlanmadı.")
        return

    table = document.add_table(rows=1, cols=len(table_data[0]))
    table.style = 'Table Grid'
    
    for i, header in enumerate(table_data[0]):
        table.cell(0, i).text = header.strip()

    for row_data in table_data[1:]:
        cells = table.add_row().cells
        for i, cell_data in enumerate(row_data):
            cells[i].text = cell_data.strip()

    print("Tablo belgeye eklendi.")

def add_content_to_document(document, content):
    if not content:
        print("Eklenecek içerik yok.")
        return

    lines = content.split('\n')
    i = 0
    while i < len(lines):
        line = lines[i].strip()
        if line.lower().startswith("başlık:"):
            title = line.replace("başlık:", "", 1).strip()
            document.add_heading(title, level=1)
            print(f"Başlık eklendi: {title}")
        elif line.lower().startswith("tablo:"):
            table_data = []
            i += 1
            while i < len(lines) and lines[i].strip() and not lines[i].strip().lower().startswith(("başlık:", "tablo:")):
                table_data.append(lines[i].strip().split(';'))
                i += 1
            if table_data:
                add_table_to_document(document, table_data)
            continue  # while döngüsünde i zaten artırıldı
        elif line:
            document.add_paragraph(line)
            print(f"Paragraf eklendi: {line[:50]}...")
        i += 1

def create_document_and_convert_to_pdf(content, output_word_file, output_pdf_file):
    try:
        document = Document()
        document.add_heading('Oluşturulan Belge', 0)

        add_content_to_document(document, content)

        document.save(output_word_file)
        print(f"Word belgesi '{output_word_file}' oluşturuldu.")

        try:
            convert(output_word_file, output_pdf_file)
            print(f"PDF dosyası '{output_pdf_file}' oluşturuldu.")
        except Exception as e:
            print(f"PDF'e dönüştürürken hata oluştu: {e}")
    except Exception as e:
        print(f"Belge oluşturulurken hata oluştu: {e}")

def main():
    text_generator = initialize_text_generator()

    prompt = """Başlık: Blockchain'in Finans Sektörüne Etkisi
    Tablo:
    Avantaj; Zorluk
    Şeffaflık; Ölçeklenebilirlik
    Güvenlik; Düzenleyici sorunlar
    Merkeziyetsizlik; Yüksek enerji tüketimi
    
    Lütfen bu noktaların her birini ve finans sektörü için etkilerini açıklayın."""

    generated_content = generate_content_with_huggingface(text_generator, prompt, max_length=1000)

    if generated_content:
        output_word_file = "olusturulan_belge_hf.docx"
        output_pdf_file = "olusturulan_belge_hf.pdf"
        create_document_and_convert_to_pdf(generated_content, output_word_file, output_pdf_file)
    else:
        print("İçerik oluşturulamadı, işlem sonlandırıldı.")

if __name__ == "__main__":
    main()

Device set to use cpu
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Metin üretme modeli başarıyla yüklendi.
İçerik başarıyla oluşturuldu.
Başlık eklendi: Başlık: Blockchain'in Finans Sektörüne Etkisi
Belge oluşturulurken hata oluştu: 'Tablo:' is not in list
